In [ ]:
from datetime import datetime
from dotenv import load_dotenv
from os import getenv
load_dotenv()
PREDICT_BUCKET = getenv("PREDICT_BUCKET") # PREDICT STUDY bucket

In [ ]:
!chmod +x pwDiffExp.R

In [ ]:
task_description = {
    "FG13": {
        "SROBJFILE": f"gs://{PREDICT_BUCKET}/Tiered-FGID-13/FG13/T0C0.rds",
        "METAFILE": f"gs://{PREDICT_BUCKET}/Tiered-FGID-13/20200630_clmeta_FG13_all.tsv",
        "COLUMN": f"curatedname"
    },
    "CD14": {
        "SROBJFILE": f"gs://{PREDICT_BUCKET}/Tiered-CD-14/T0C0.rds",
        "METAFILE": f"gs://{PREDICT_BUCKET}/Tiered-CD-14/20200630_clmeta_CD14_all.tsv",
        "COLUMN": f"curatedname"
    }
}

In [ ]:
task_description

In [ ]:
for k, v in task_description.items():
    !dsub \
        --name pw-de-{k} \
        --provider google-v2 \
        --project shalek-lab \
        --logging gs://{PREDICT_BUCKET}/pwdiffexp/logging \
        --regions us-central1 \
        --boot-disk-size 15  --timeout 2w \
        --min-ram 256 --min-cores 32 --disk-size 500 \
        --input SROBJFILE={v['SROBJFILE']} \
        --input METAFILE={v['METAFILE']} \
        --env COLUMN={v['COLUMN']} \
        --output OUTRDSFILE=gs://{PREDICT_BUCKET}/pwdiffexp/{datetime.now().strftime('%Y%m%d-%H%M%S')}_{k}_pwde_{v['COLUMN']}.rds \
        --image shaleklab/terra-seurat-env:release-v2.0 \
        --script pwDiffExp.R